In [2]:
import pandas as pd

def fasta_to_dataframe(fasta_file):
    headers = []
    sequences = []
    with open(fasta_file, 'r') as f:
        current_sequence = []
        current_header = None
        for line in f:
            line = line.strip()
            if line.startswith('>'):  # Header line
                if current_header is not None:
                    # Save the previous sequence
                    headers.append(current_header)
                    sequences.append(''.join(current_sequence))
                current_header = line[1:]  # Remove '>'
                current_sequence = []
            else:
                current_sequence.append(line)
        # Add the last sequence
        if current_header is not None:
            headers.append(current_header)
            sequences.append(''.join(current_sequence))
    # Create a DataFrame
    return pd.DataFrame({'Header': headers, 'Sequence': sequences})

In [4]:
fasta_file = "../data/DB.COX1.fna"
df = fasta_to_dataframe(fasta_file)

In [12]:
search_term = 'o:Entomobryomorpha'

count = df['Header'].str.contains(search_term).sum()
print(f"Number of headers containing {search_term}: {count}")

Number of headers containing o:Entomobryomorpha: 50398
